In [1]:
kgDir: str = "../data/kg_embedding";
import os;
assert os.path.exists(kgDir);

import subprocess;
from typing import List;
files: List[str] = subprocess.Popen(f"ls {kgDir}", shell=True, stdout=subprocess.PIPE).stdout.read().decode()[:-1].split("\n");

def __service_compFileUri(base: str, fname: str) -> str:
    tar: str = f"{kgDir}{'/' if kgDir[-1] != '/' else ''}{fname}";
    assert os.path.exists(tar);
    return tar;

fList: List[str] = [__service_compFileUri(kgDir, f) for f in files];
print(fList);

['../data/kg_embedding/disease_embedding_cui.txt', '../data/kg_embedding/disease_embedding_icd10.txt', '../data/kg_embedding/disease_embedding_icd9.txt', '../data/kg_embedding/drug_embedding.txt']


In [2]:
import numpy as np;
from typing import Dict;

def __service_fetchMappingDis(fname: str, d: Dict[str, np.ndarray]) -> None:
    assert os.path.exists(fname);
    with open (fname, "r") as f:
        cont: List[str] = f.read().split("\n");
        if cont[-1] == "":
            cont = cont[:-1];
        if len(cont[0].split("\t")) < 3:
            return;
        cont = cont[1:];
        for c in cont:
            _, icd, emb = c.split("\t");
            d[icd[2:-2].replace(".", "")] = np.array([float(ele) for ele in emb[1:-1].split(", ")]);

def __service_fetchMappingDrug(fname: str, d: Dict[str, np.ndarray]) -> None:
    assert os.path.exists(fname);
    with open (fname, "r") as f:
        cont: List[str] = f.read().split("\n");
        if cont[-1] == "":
            cont = cont[:-1];
        if len(cont[0].split("\t")) < 2:
            return;
        cont = cont[1:];
        for c in cont:
            db, emb = c.split("\t");
            d[db] = np.array([float(ele) for ele in emb[1:-1].split(", ")]);

In [5]:
icd2emb: Dict[str, np.ndarray] = dict();
dbk2emb: Dict[str, np.ndarray] = dict();

for f in fList:
    if f.split("/")[-1].startswith("disease"):
        __service_fetchMappingDis(f, icd2emb);
    elif f.split("/")[-1].startswith("drug"):
        __service_fetchMappingDrug(f, dbk2emb);
    else:
        print(f"No processor found for {f}");

import pickle;
with open("../data/kgEmb.pkl", "wb") as f:
    pickle.dump((icd2emb, dbk2emb), f);